In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datasetsforecast.long_horizon import LongHorizon

from neuralforecast.core import NeuralForecast
from neuralforecast.models import NHITS, iTransformer, TSMixer

from utilsforecast.losses import mae, mse
from utilsforecast.evaluation import evaluate

D:\Anaconda\envs\neuralforecast-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_data(name):
    if name == "ettm1":
        Y_df, *_ = LongHorizon.load(directory='./', group='ETTm1')
        Y_df['ds'] = pd.to_datetime(Y_df['ds'])
        val_size = 11520
        test_size = 11520
        freq = '15T'
    elif name == "ettm2":
        Y_df, *_ = LongHorizon.load(directory='./', group='ETTm2')
        Y_df['ds'] = pd.to_datetime(Y_df['ds'])
        val_size = 11520
        test_size = 11520
        freq = '15T'
    elif name == 'etth1':
        Y_df, *_ = LongHorizon.load(directory='./', group='ETTh1')
        Y_df['ds'] = pd.to_datetime(Y_df['ds'])
        val_size = 2880
        test_size = 2880
        freq = 'H'
    elif name == "etth2":
        Y_df, *_ = LongHorizon.load(directory='./', group='ETTh2')
        Y_df['ds'] = pd.to_datetime(Y_df['ds'])
        val_size = 2880
        test_size = 2880
        freq = 'H'

    return Y_df, val_size, test_size, freq

In [4]:
Y_df, val_size, test_size, freq = load_data('ettm1')
horizon = 96

Y_df.head()

,unique_id,ds,y
0,HUFL,2016-07-01 00:00:00,-0.370818
1,HUFL,2016-07-01 00:15:00,-0.382304
2,HUFL,2016-07-01 00:30:00,-0.382304
3,HUFL,2016-07-01 00:45:00,-0.382304
4,HUFL,2016-07-01 01:00:00,-0.393790


In [5]:
Y_df['unique_id'].unique()

array(['HUFL', 'HULL', 'LUFL', 'LULL', 'MUFL', 'MULL', 'OT'], dtype=object)

In [7]:
models = [
    # iTransformer
    iTransformer(
        h=horizon,
        input_size=3*horizon,
        n_series=7,
        hidden_size=512,
        n_heads=8,
        e_layers=2,
        d_layers=1,
        d_ff=1024,
        use_norm=True,
        max_steps=1000,
        early_stop_patience_steps=3),
    )
    TSMixer(h=horizon, input_size=3*horizon, n_series=7, max_steps=1000, early_stop_patience_steps=3),
    NHITS(h=horizon, input_size=3*horizon, max_steps=1000, early_stop_patience_steps=3),
]

nf = NeuralForecast(models=models, freq=freq)
nf_preds = nf.cross_validation(df=Y_df, val_size=val_size, test_size=test_size, n_windows=None)
nf_preds = nf_preds.reset_index()

ettm1_evaluation = evaluate(df=nf_preds, metrics=[mae, mse], models=['iTransformer', 'TSMixer', 'NHITS'])

INFO:lightning_fabric.utilities.seed:Global seed set to 1
INFO:lightning_fabric.utilities.seed:Global seed set to 1
INFO:lightning_fabric.utilities.seed:Global seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type                   | Params
---------------------------------------------------------
0 | loss          | MAE                    | 0     
1 | padder        | ConstantPad1d          | 0     
2 | scaler        | TemporalNorm           | 0     
3 | enc_embedding | DataEmbedding_inverted | 147 K 
4 | encoder       | TransEncoder           | 4.2 M 
5 | projector     | Linear                 | 49.2 K
-------------------------------------

Epoch 99: 100%|█████████████████| 1/1 [00:00<00:00,  4.74it/s, v_num=48, train_loss_step=0.345, train_loss_epoch=0.357]
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                                                   | 0/1 [00:00<?, ?it/s]
Epoch 99: 100%|█| 1/1 [00:03<00:00,  3.55s/it, v_num=48, train_loss_step=0.345, train_loss_epoch=0.357, valid_loss=0.43
Epoch 199: 100%|█| 1/1 [00:00<00:00,  5.62it/s, v_num=48, train_loss_step=0.363, train_loss_epoch=0.371, valid_loss=0.4
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                                                   | 0/1 [00:00<?, ?it/s]
Epoch 199: 100%|█| 1/1 [00:03<00:00,  3.48s/it, v_num=48, train_loss_step=0.363, train_loss_epoch=0.371, valid_loss=0.4
Epoch 299: 100%|█| 1/1 [00:00<00:00,  5.75it/s, v_num=48, train_loss_step=0.383, train_loss_epoch=0.351, valid_loss=0.4
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                     

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1000` reached.


Epoch 999: 100%|█| 1/1 [00:03<00:00,  3.35s/it, v_num=48, train_loss_step=0.351, train_loss_epoch=0.351, valid_loss=0.4

INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


...


Predicting DataLoader 0: 100%|███████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.44s/it]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type          | Params
-----------------------------------------------
0 | loss         | MAE           | 0     
1 | padder_train | ConstantPad1d | 0     
2 | scaler       | TemporalNorm  | 0     
3 | blocks       | ModuleList    | 3.2 M 
-----------------------------------------------
3.2 M     Trainable params
0         Non-trainable params
3.2 M     Total params
12.759    Total estimated model params size (MB)



Epoch 99: 100%|█████████████████| 1/1 [00:00<00:00,  2.27it/s, v_num=52, train_loss_step=0.327, train_loss_epoch=0.335]
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                                                   | 0/1 [00:00<?, ?it/s]
Epoch 99: 100%|█| 1/1 [00:02<00:00,  2.78s/it, v_num=52, train_loss_step=0.327, train_loss_epoch=0.335, valid_loss=0.40
Epoch 199: 100%|█| 1/1 [00:00<00:00,  1.78it/s, v_num=52, train_loss_step=0.299, train_loss_epoch=0.314, valid_loss=0.4
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                                                   | 0/1 [00:00<?, ?it/s]
Epoch 199: 100%|█| 1/1 [00:02<00:00,  2.66s/it, v_num=52, train_loss_step=0.299, train_loss_epoch=0.314, valid_loss=0.4
Epoch 299: 100%|█| 1/1 [00:00<00:00,  1.32it/s, v_num=52, train_loss_step=0.296, train_loss_epoch=0.299, valid_loss=0.4
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|                                    

INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|███████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.94s/it]


In [8]:
ettm1_evaluation

,unique_id,metric,iTransformer,TSMixer,NHITS
0,HUFL,mae,0.541420,0.478587,0.498946
1,HULL,mae,0.325672,0.306731,0.318453
2,LUFL,mae,0.477560,0.437289,0.455887
3,LULL,mae,0.250314,0.237742,0.243399
4,MUFL,mae,0.541698,0.476821,0.492531
5,MULL,mae,0.289053,0.272388,0.283276
6,OT,mae,0.136055,0.129631,0.139908
7,HUFL,mse,0.623118,0.564179,0.620087
8,HULL,mse,0.187513,0.172206,0.188316
9,LUFL,mse,0.489639,0.439848,0.471323


In [9]:
ettm1_evaluation = ettm1_evaluation.drop(['unique_id'], axis=1)\
                                   .groupby('metric')\
                                   .mean()\
                                   .reset_index()

ettm1_evaluation

,metric,iTransformer,TSMixer,NHITS
0,mae,0.365967,0.334170,0.347486
1,mse,0.321492,0.292147,0.317878
